In [1]:
BASE_URL = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=2330"

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def get_basic_info_selenium(stock_id):
    url = f"https://goodinfo.tw/tw/BasicInfo.asp?STOCK_ID={stock_id}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 不開啟視窗
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)  # 等待 JS 載入真正內容

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # 解析基本資料
    info = {}
    rows = soup.find_all("tr")
    for row in rows:
        ths = row.find_all("th", class_="bg_h1")
        tds = row.find_all("td", bgcolor="white")
        for th, td in zip(ths, tds):
            key = th.get_text(strip=True).replace("\u3000", "")
            val = td.get_text(strip=True)
            if key and val:
                info[key] = val

    return info

info = get_basic_info_selenium("2330")
for k, v in info.items():
    print(f"{k}：{v}")

股票代號：2330
股票名稱：台積電
產業別：半導體業
上市/上櫃：上市
公司名稱：台灣積體電路製造股份有限公司
英文簡稱：TSMC
成立日期：1987/02/21(38.3年)
掛牌日期：1994/09/05(30.8年)
上市日期：1994/09/05
上櫃日期：-
興櫃日期：-
公開發行日期：1992/10/30
資本額：2,593.26億元
每股面值：新台幣 10元
目前市值：25.8兆
公司債發行：有
發行股數：25,932,615,521股(含私募0股)
特別股：0股
財報編製類型：合併
盈餘分派頻率：每季
董事長：魏哲家
總經理：總裁: 魏哲家
發言人：黃仁昭
發言人職稱：資深副總經理暨財務長
代理發言人：高孟華
統一編號：22099131
總機電話：03-5636688
傳真電話：03-5797337
公司電郵：invest@tsmc.com
公司網址：https://www.tsmc.com
中文地址：新竹科學園區力行六路8號
英文地址：No. 8, Li-Hsin Rd. 6, Hsinchu Science Park,
Hsin-Chu 300096, Taiwan, R.O.C.
主要業務：依客戶之訂單與其提供之產品設計說明，以從事製造與銷售積體電路以及其他晶圓半導體裝置。提供前述產品之封裝與測試服務、積體電路之電腦輔助設計技術服務。提供製造光罩及其設計服務。
股票過戶機構：中國信託商業銀行 代理部
過戶機構電話：02-6636-5566
過戶機構地址：台北市重慶南路一段83號5樓
簽證會計師事務所：勤業眾信聯合會計師事務所
簽證會計師：吳世宗, 林尚志
投資人關係聯絡人：蘇志凱
聯絡人職稱：處長
投資人關係聯絡電話：03-563-6688
聯絡人電子郵件：jeff_su@tsmc.com
利害關係人專區網址：https://esg.tsmc.com/zh-Hant/sustainable-management/materiality-analysis


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_full_kline_data(stock_id="2330"):
    url = f"https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={stock_id}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    table = soup.find("table", id="tblDetail")
    if not table:
        raise Exception("找不到 tblDetail 表格 ─ Goodinfo 可能改版了")

    rows = table.find_all("tr")
    data = []
    for row in rows:
        cols = row.find_all("td")
        values = [td.text.strip().replace(",", "").replace("\xa0", "") for td in cols if td.text.strip()]
        if len(values) >= 20:  # 避免標題列或空列
            data.append(values)

    # 建立 DataFrame，欄位名稱參考截圖
    columns = [
        "日期", "開盤", "最高", "最低", "收盤", "漲跌", "漲跌幅", "振幅(%)",
        "張數", "筆數", "均價", "億元",
        "外資", "投信", "自營", "合計", "外資持股(%)",
        "融資增減", "融資餘額", "融券增減", "融券餘額", "券資比(%)"
    ]
    df = pd.DataFrame(data, columns=columns[:len(data[0])])  # 避免欄位數對不上

    return df

# 使用示範
df = get_full_kline_data("2330")
print(df.head())


          日期    開盤    最高   最低   收盤   漲跌    漲跌幅 振幅(%)     張數      筆數  ...  \
0  '25/06/06   997   997  991  995   -3   -0.3   0.6  18797   27283  ...   
1  '25/06/05  1000  1000  991  998   +8  +0.81  0.91  27048   51309  ...   
2  '25/06/04   974   990  970  990  +40  +4.21  2.11  43196   73539  ...   
3  '25/06/03   960   965  950  950   +4  +0.42  1.59  27483   43550  ...   
4  '25/06/02   958   961  946  946  -21  -2.17  1.55  40608  125245  ...   

       外資     投信    自營      合計 外資持股(%)   融資增減   融資餘額  融券增減 融券餘額 券資比(%)  
0   +1004   +238  +165   +1407    72.6   -149  19806  -384   12   0.06  
1   +9823   +689  -725   +9788    72.7   -424  19955  -167  396   1.98  
2  +15827   +479  +462  +16768    72.7  -1185  20379  -181  563   2.76  
3    -656   +146  -354    -863    72.6   -148  21564    -2  744   3.45  
4  -13196  +23.6  -586  -13759    72.6   +314  21712   -51  746   3.44  

[5 rows x 22 columns]
